In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nest

from scripts import visualizations, initializations, optimization, experiments, model, neuron_categorization



              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.6.0
 Built: Sep 28 2023 11:53:34

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



In [2]:
df = pd.read_csv('/hpc/mzhu843/modelling/nest/DATA/raw/ezytrack/behaviour_LocationOutput.csv')
spikes = np.load('/hpc/mzhu843/modelling/nest/DATA/processed/spikes/S_3_p.npy')

categorizer = neuron_categorization.NeuronCategorizer(spikes, df)

categorizer.run_categorization()

categorized_neurons = categorizer.get_categorized_neurons()

neuron_id = '1'
neuron_spike_coords = categorized_neurons['Place'][neuron_id]

categorizer.print_category_counts()
categorizer.save_place_fields_box()





Number of place cells: 90
Number of silent cells: 115
Number of interneurons: 1


IndexError: index 131 is out of bounds for axis 0 with size 129